In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
sns.set_style("whitegrid")

First we load the data (data comes from cleaning the full dataset; see Clean_Full_Dataset.ipynb for details), drop any rows with NAs, and convert the 'CheckoutDate' column to a DateTime datatype. We also sort by CheckoutDate and reset the index; this step is unnecessary but helpful for viewing the data in chronological checkout order. 

In [2]:
#Read in cleaned data
#df=pd.read_csv('data/Checkouts_2005_cleaned.csv')
df=pd.read_csv('data/Checkouts_2005_cleaned_w_colon.csv')
df = df.dropna()
df['CheckoutDate'] = pd.to_datetime(df['CheckoutDate'])
df = df.sort_values(by='CheckoutDate')
df =df.reset_index(drop=True)

df.head()

,UsageClass,MaterialType,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CleanedTitle,CleanedCreator,CheckoutDate
0,Physical,BOOK,3,The rosary girls / Richard Montanari.,"Montanari, Richard","Balzano Jessica Fictitious character Fiction, ...","Ballantine Books,",2005,the rosary girls,montanari richard,2005-04-01
1,Physical,BOOK,2,Secrets of the millionaire mind : mastering th...,"Eker, T. Harv","Money Psychological aspects, Millionaires Psyc...","HarperBusiness,",2005,secrets of the millionaire mind : mastering th...,eker harv t,2005-04-01
2,Physical,BOOK,7,In the company of liars / David Ellis.,"Ellis, David, 1967-",Thrillers Fiction,"G.P. Putnam's Sons,",2005,in the company of liars,david ellis,2005-04-01
3,Physical,BOOK,21,The motive / John Lescroart.,"Lescroart, John T.","Hardy Dismas Fictitious character Fiction, Soc...","Dutton,",2005,the motive,john lescroart t,2005-04-01
4,Physical,BOOK,1,The rottweiler [text (large print)] / Ruth Ren...,"Rendell, Ruth, 1930-2015","Police England London Fiction, Lisson Grove Lo...","Thorndike Press,",2005,the rottweiler,rendell ruth,2005-04-01


Our goal is to make a new dataset where each row corresponds to a unique set of
(CleanedTitle, CleanedCreator, UsageClass, MaterialType). The dataframe will have a column for each month of checkout data we have (April 2005 - August 2024), where these columns contain the number of checkouts in that month for each book respectively.

We first create a list of the months we have checkout data from, where each month is a string of the form MM/DD/YYYY. This list of strings will be used to populate the columns of our dataframe later.

In [3]:
#Get a list of the months we have checkout data from 
months_of_interest=pd.date_range(start='2005-04-01', end='2024-08-01',freq='MS')


In [4]:
print(months_of_interest)

DatetimeIndex(['2005-04-01', '2005-05-01', '2005-06-01', '2005-07-01',
               '2005-08-01', '2005-09-01', '2005-10-01', '2005-11-01',
               '2005-12-01', '2006-01-01',
               ...
               '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01',
               '2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01',
               '2024-07-01', '2024-08-01'],
              dtype='datetime64[ns]', length=233, freq='MS')


In [8]:
#Get the list of all months as strings; will be used as column names later
month_columns = months_of_interest.strftime('%m/%d/%Y')
month_columns = month_columns.to_list()


The dataset has a wide variety of Material Types: 

['BOOK' 'SOUNDDISC' 'SOUNDREC' 'SOUNDDISC, VIDEODISC' 'MUSIC' 'AUDIOBOOK'
 'EBOOK' 'VIDEODISC' 'REGPRINT' 'ER' 'VISUAL' 'ATLAS' 'MAP' 'SOUNDCASS'
 'VIDEO' 'ER, MAP' 'ER, SOUNDDISC' 'ER, SOUNDDISC, VIDEODISC'
 'MUSICSNDREC' 'FLASHCARD, SOUNDDISC' 'ER, PRINT' 'ER, NONPROJGRAPH'
 'ER, VIDEODISC' 'LARGEPRINT' 'NOTATEDMUSIC' 'MAP, VIEW'
 'REGPRINT, VIDEOREC' 'KIT' 'REGPRINT, SOUNDDISC' 'ER, REGPRINT'
 'FLASHCARD' 'UNSPECIFIED' 'MIXED' 'BOOK, ER' 'ER, SOUNDREC'
 'ER, SOUNDDISC, SOUNDREC' 'PHOTO']

 To simplify our analysis, we take the five categories with the most checkouts and categorize all other types as 'OTHER'. 

In [9]:
#Valid material types
material_types = ['BOOK','EBOOK','SOUNDDISC','VIDEODISC','AUDIOBOOK', 'OTHER']


In [10]:
#Print original list of MaterialTypes found in dataset
print(df.MaterialType.unique())
#Filter out the material types not in material_types and replace them with "OTHER"
df['MaterialType'] = df['MaterialType'].where(df['MaterialType'].isin(material_types), 'OTHER')
#Print the new list of MaterialTypes found in dataset
print(df.MaterialType.unique())

['BOOK' 'SOUNDDISC' 'OTHER' 'AUDIOBOOK' 'EBOOK' 'VIDEODISC']
['BOOK' 'SOUNDDISC' 'OTHER' 'AUDIOBOOK' 'EBOOK' 'VIDEODISC']


In [8]:
#Sample dataframe used for priliminary testing
df_sample = df.sample(1000)
df_sample = df_sample.sort_values(by='CheckoutDate')
df_sample =df_sample.reset_index(drop=True)
df_sample.head()

,UsageClass,MaterialType,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CleanedTitle,CleanedCreator,CheckoutDate
0,Physical,BOOK,1,Echoes of fury : the 1980 eruption of Mount St...,"Parchman, Frank","Saint Helens Mount Wash Eruption 1980, Saint H...",Epicenter Press : Distributed by Graphic Arts ...,2005,echoes of fury,frank parchman,2005-04-01
1,Physical,BOOK,3,Our Eleanor : a scrapbook look at Eleanor Roos...,"Fleming, Candace",Roosevelt Eleanor 1884 1962 Juvenile literatur...,"Atheneum Books for Young Readers,",2005,our eleanor,candace fleming,2006-02-01
2,Physical,BOOK,1,Soccer rules explained / Stanley Lover ; forew...,"Lover, Stanley F.",Soccer Rules,"Lyons Press,",2005,soccer rules explained,f lover stanley,2006-02-01
3,Physical,SOUNDDISC,22,The greatest [sound recording] / Cat Power.,"Cat Power, 1972-","Rock music 2001 2010, Popular music 2001 2010","Matador,",2006,the greatest,cat power,2006-06-01
4,Digital,AUDIOBOOK,1,Day of the Dead: Sheriff Brandon Walker Myster...,J. A. Jance,"Fiction, Mystery",Books In Motion,2005,day of the dead,a j jance,2006-08-01


We get the unique sets from (CleanedTitle, CleanedCreator, UsageClass, MaterialType) using value_counts(). We see that we have 621171 (715720 in the case we don't filter out colon parts of title) different possible combinations of these features out of the ~25 million rows in the dataframe. Note that when we only used unique pairs of (CleanedTitle, CleanedCreator), we had around 440,000 different combinations. 

In [12]:
#Get the unique sets from (CleanedTitle, CleanedCreator, UsageClass, MaterialType)
pairs = df[['CleanedTitle', 'CleanedCreator','UsageClass', 'MaterialType']].value_counts().reset_index()
pairs = pairs.drop(columns='count')
    
pairs.shape

(715720, 4)

We create a numpy array to store the checkout data for each month with rows corresponding to the rows in pairs and columns corresponding to the month in month_columns. We will then convert this to a dataframe to be merged with pairs later, which is significantly faster than updating pairs along the way. 

In [13]:
#Create numpy array of zeros to store the number of checkouts in each month
#Rows: indicate corresponding row in pairs
#Columns: indicate corresponding month in month_columns
months_count = np.zeros((pairs.shape[0], len(month_columns)),dtype = int)

In [14]:
#Get the column names of the features we want to add back in
features = df.columns.to_list()

#Remove the column names we already have in pairs
features.remove('CleanedTitle')
features.remove('CleanedCreator')
features.remove('UsageClass')
features.remove('MaterialType')


print(features)

['Checkouts', 'Title', 'Creator', 'Subjects', 'Publisher', 'PublicationYear', 'CheckoutDate']


The main processing is done in the cell below: 
## Preprocessing: 
To speed up the runtime, we create a few columns with corresponding dictionaries that will enable us to grab row/column indices without searching through the dataframe. We use dictionaries because they are implemented via a hash map so that the average lookup time is O(1), compared with the O(N) we would need if we searched through the dataframe directly. The 'CheckoutDate_str' column converts the 'CheckoutDate' to a string that matches the month strings in month_columns; the dictionary date_to_idx will allow us to go from this string to the appropriate column index of the month we want to update in month_counts. The column 'index_str' contains the unique combination of the (CleanedTitle, CleanedCreator, UsageClass, MaterialType) converted to a single string separated by commas. This simplifies the processing so that we only need to consider one column from the row we have instead of four. We use this column and the indices in pairs to create a dictionary pairs_lookup which will allow us to go from the 'index_str' in df to the corresponding row index in pairs that it corresponds to. This index will be used to update month_counts and the other feature data for each row in pairs. Finally we initialize an array of zeros called filled, which signifies if the additional data corresponding to the column names found in features has been found for each row in pairs. Since df is sorted by CheckoutDate and .apply() is applied top down (first row to last row), we should grab the first CheckoutDate for the book and use its features information to update the row. Note that there is no guarentee that all books of type (CleanedTitle, CleanedCreator, UsageClass, MaterialType) have the same Publisher, Subject, PublicationYear, etc., but they should be similar so we choose to grab the first one. 

## Main Processing: 
updates_months_full is applied to each row of our dataframe df via df.apply(). As it traverses each row, it updates month_counts month index and pairs index it obtains from the date_to_idx and pairs_lookup dictionaries based on the 'CheckoutDate_str' and 'index_str' columns of the row. Additionally, it checks if we have found the additional data corresponding to the column names in features by checking the appropriate index in filled. If not, we return the features values from the row in addition to the row index of pairs. 

## Output: 
After applying updates_months_full to each row in df, the months_count array will contain checkout data from each month for each row in pairs. Additionally, for each row in pairs, updates will contain the corresponding features column information as a list of lists. 

In [15]:
# Convert 'CheckoutDate' to string in the desired format beforehand
df['CheckoutDate_str'] = df['CheckoutDate'].dt.strftime('%m/%d/%Y')

#Create columns for easier lookup in pairs and df; we treat this column as an index column 
pairs['index_str'] = pairs['CleanedTitle'] + " , " + pairs['CleanedCreator'] + " , " \
    + pairs['UsageClass'] + " , " + pairs['MaterialType']

df['index_str'] =  df['CleanedTitle'] + " , " + df['CleanedCreator'] + " , " \
                 + df['UsageClass'] + " , " + df['MaterialType']

#Create dictionary with keys given by index_str and values the corresponding index in pairs
vals = pairs['index_str'].values
indices = pairs.index
pairs_lookup = dict(zip(vals, indices))

#Dictionary to go from CheckoutDate_str to index in list of months
date_to_idx = {date_str: idx for idx, date_str in enumerate(month_columns)}



#Create array indicating which rows of pairs we have already filled in
# 1 indicates we already filled it; 0 indicates we have not
filled= [0] * len(indices)

#Function to aggregate all the needed info
def update_months_full(row):
    #Update months count with checkout information
    date_idx  = date_to_idx[row['CheckoutDate_str']]
    pairs_idx = pairs_lookup[row['index_str']]
    months_count[pairs_idx, date_idx] +=  row['Checkouts']

    #Update additional information 
    title_creator_str = row['index_str']
    #Check if we have updated the row yet; if not, update it
    if filled[pairs_lookup[title_creator_str]]==0:
        #Change indicator to 1 to indicate we have visited row
        filled[pairs_lookup[title_creator_str]]=1
        #Return the index in pairs with the corresponding features from df
        return list(np.insert(row[features].values, 0, pairs_lookup[title_creator_str]))
    return None



# Apply the update_row function to each row in df
updates = df.apply(update_months_full, axis=1)
updates = updates.dropna()
assert(updates.shape[0]==pairs.shape[0])
updates.head()



0    [154013, 3, The rosary girls / Richard Montana...
1    [1753, 2, Secrets of the millionaire mind : ma...
2    [55184, 7, In the company of liars / David Ell...
3    [20236, 21, The motive / John Lescroart., Lesc...
4    [142696, 1, The rottweiler [text (large print)...
dtype: object

In [16]:
#Make sure pairs and updates have the same number of rows
print(pairs.shape)
print(updates.shape)


(715720, 5)
(715720,)


Now that we have all the information we need, we will create dataframes for updates and month_counts and merge them with pairs. We create dataframes via pd.DataFrame on month_counts and updates. updates already contains an index column which corresponds to the indices in pairs. To merge on these indices, we create corresponding 'index' columns in pairs and months_df via reset_index(). Note that the indicies of months_df already match those of pairs due to the construction of month_counts.  Finally, we merge updates_df with pairs and then the result of that with month_df on the index column to obtain the final dataframe merged_df. We drop some of the extra columns we created for processing and finally write the data to the file cleaned_months_with_types.csv in the data folder. 

In [17]:
# Features we obtained in updates 
# We added a column for the corresponding index of pairs, so we add the column name here
features_new = ['index', *features]
print(features_new)

['index', 'Checkouts', 'Title', 'Creator', 'Subjects', 'Publisher', 'PublicationYear', 'CheckoutDate']


In [18]:
#Remove the extra column we made for indexing convenience
pairs = pairs.drop(columns=['index_str'])

#Add column with index of each row in pairs for merging 
pairs = pairs.reset_index()
pairs.head()

,index,CleanedTitle,CleanedCreator,UsageClass,MaterialType
0,0,the walking dead.,kirkman robert,Physical,BOOK
1,1,ranma 1,rumiko takahashi,Physical,BOOK
2,2,gantz.,hiroya oku,Physical,BOOK
3,3,rin-ne.,rumiko takahashi,Physical,BOOK
4,4,pandora hearts.,jun mochizuki,Physical,BOOK


In [19]:
#Create updates dataframe to be joined with pairs
#The index column corresponds to the index column of pairs
updates_df =pd.DataFrame(updates.to_list(), columns=features_new)
updates_df.head()



,index,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CheckoutDate
0,154013,3,The rosary girls / Richard Montanari.,"Montanari, Richard","Balzano Jessica Fictitious character Fiction, ...","Ballantine Books,",2005,2005-04-01
1,1753,2,Secrets of the millionaire mind : mastering th...,"Eker, T. Harv","Money Psychological aspects, Millionaires Psyc...","HarperBusiness,",2005,2005-04-01
2,55184,7,In the company of liars / David Ellis.,"Ellis, David, 1967-",Thrillers Fiction,"G.P. Putnam's Sons,",2005,2005-04-01
3,20236,21,The motive / John Lescroart.,"Lescroart, John T.","Hardy Dismas Fictitious character Fiction, Soc...","Dutton,",2005,2005-04-01
4,142696,1,The rottweiler [text (large print)] / Ruth Ren...,"Rendell, Ruth, 1930-2015","Police England London Fiction, Lisson Grove Lo...","Thorndike Press,",2005,2005-04-01


In [20]:
#Create months dataframe containing the counts for each month; to be joined with pairs
# Note arrays index with (0,0) as top left so the index column is aligned with the index
# columns of pairs
months_df = pd.DataFrame(months_count, columns=month_columns)
#Add index column to match with pairs
months_df = months_df.reset_index()
months_df.head()


,index,04/01/2005,05/01/2005,06/01/2005,07/01/2005,08/01/2005,09/01/2005,10/01/2005,11/01/2005,12/01/2005,...,11/01/2023,12/01/2023,01/01/2024,02/01/2024,03/01/2024,04/01/2024,05/01/2024,06/01/2024,07/01/2024,08/01/2024
0,0,0,0,0,0,0,0,0,0,0,...,5,5,6,12,15,1,0,0,4,2
1,1,0,0,0,0,0,0,0,0,0,...,24,5,5,10,24,6,0,0,4,4
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,1,3,1,20,5,1,0,0,2,7
4,4,0,0,0,0,0,0,0,0,0,...,4,5,0,0,0,2,0,0,0,0


In [27]:
merged1_df = pd.merge(updates_df, pairs, on='index')
merged_df = pd.merge(merged1_df, months_df, on='index')

#Sort by index so that when 
merged_df = merged_df.sort_values(by='index')
#Remove the index and Checkouts columns as they are unnecessary. 
merged_df = merged_df.drop(columns=['index', 'Checkouts'])
merged_df.head()


,Title,Creator,Subjects,Publisher,PublicationYear,CheckoutDate,CleanedTitle,CleanedCreator,UsageClass,MaterialType,...,11/01/2023,12/01/2023,01/01/2024,02/01/2024,03/01/2024,04/01/2024,05/01/2024,06/01/2024,07/01/2024,08/01/2024
0,The rosary girls / Richard Montanari.,"Montanari, Richard","Balzano Jessica Fictitious character Fiction, ...","Ballantine Books,",2005,2005-04-01,the rosary girls,montanari richard,Physical,BOOK,...,0,0,0,0,0,0,0,0,0,0
1,Secrets of the millionaire mind : mastering th...,"Eker, T. Harv","Money Psychological aspects, Millionaires Psyc...","HarperBusiness,",2005,2005-04-01,secrets of the millionaire mind : mastering th...,eker harv t,Physical,BOOK,...,2,0,1,1,2,1,0,0,0,2
2,In the company of liars / David Ellis.,"Ellis, David, 1967-",Thrillers Fiction,"G.P. Putnam's Sons,",2005,2005-04-01,in the company of liars,david ellis,Physical,BOOK,...,0,0,0,0,0,0,0,0,0,0
3,The motive / John Lescroart.,"Lescroart, John T.","Hardy Dismas Fictitious character Fiction, Soc...","Dutton,",2005,2005-04-01,the motive,john lescroart t,Physical,BOOK,...,0,0,0,0,0,0,0,0,0,0
4,The rottweiler [text (large print)] / Ruth Ren...,"Rendell, Ruth, 1930-2015","Police England London Fiction, Lisson Grove Lo...","Thorndike Press,",2005,2005-04-01,the rottweiler,rendell ruth,Physical,BOOK,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#Make sure data matches 
merged_df[['Title', 'Creator', 'CleanedTitle', 'CleanedCreator']].head(50)

,Title,Creator,CleanedTitle,CleanedCreator
0,The rosary girls / Richard Montanari.,"Montanari, Richard",the rosary girls,montanari richard
1,Secrets of the millionaire mind : mastering th...,"Eker, T. Harv",secrets of the millionaire mind : mastering th...,eker harv t
2,In the company of liars / David Ellis.,"Ellis, David, 1967-",in the company of liars,david ellis
3,The motive / John Lescroart.,"Lescroart, John T.",the motive,john lescroart t
4,The rottweiler [text (large print)] / Ruth Ren...,"Rendell, Ruth, 1930-2015",the rottweiler,rendell ruth
5,Shadow life : a portrait of Anne Frank and her...,"Denenberg, Barry",shadow life : a portrait of anne frank and her...,barry denenberg
6,Over under / Marthe Jocelyn ; [illustrated by]...,"Jocelyn, Marthe",over under,jocelyn marthe
7,"Jimi Hendrix : the man, the magic, the truth /...","Lawrence, Sharon, 1948-","jimi hendrix : the man, the magic, the truth",lawrence sharon
8,Cold service / Robert B. Parker.,"Parker, Robert B., 1932-2010",cold service,b parker robert
9,Eleanor Rigby : a novel / Douglas Coupland.,"Coupland, Douglas",eleanor rigby : a novel,coupland douglas


In [29]:
#merged_df.to_csv('data/cleaned_months_with_type_2005.csv', index=False)
merged_df.to_csv('data/cleaned_months_with_type_colon_2005.csv', index=False)

## Some additional exploration: 

We note that of the materials we have, most of them are physical books, followed by ebooks, audiobooks, sounddiscs. A small proportion is composed of other with both physical and digital classes and finally there are few videodiscs.

In [30]:
pairs = merged_df[['UsageClass', 'MaterialType']].value_counts().reset_index()
pairs.head(pairs.shape[0])

,UsageClass,MaterialType,count
0,Physical,BOOK,305716
1,Digital,EBOOK,247796
2,Digital,AUDIOBOOK,99619
3,Physical,SOUNDDISC,51280
4,Digital,OTHER,6115
5,Physical,OTHER,4657
6,Physical,VIDEODISC,537
